In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split


In [2]:
with open('/content/636_project1_train_images', 'rb') as f:
    images = pickle.load(f)

with open('/content/636_project1_train_labels', 'rb') as f:
    labels = pickle.load(f)


In [3]:
images = images.numpy()
labels = labels.numpy()


In [4]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.15, random_state=42)
#train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


In [5]:
train_images = train_images / 255.0 #Preprocessing the data
test_images = test_images / 255.0
#val_images = val_images / 255.0

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)
#val_labels = tf.keras.utils.to_categorical(val_labels, 10)


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


In [7]:
# Stops training when 'val_loss' has stopped decreasing for 20 epochs.
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

# Saves the best model (in terms of validation accuracy) as 'best_model.h5'.
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Reduces the learning rate when 'val_loss' has stopped decreasing.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=0.00001)


In [8]:
model = tf.keras.Sequential([
    # First Convolutional Block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # Convolutional layer
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),  # Convolutional layer with L2 regularization
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.MaxPooling2D((2, 2)),  # Max pooling
    tf.keras.layers.Dropout(0.5),  # Dropout layer

    # Second Convolutional Block
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),  # Convolutional layer with L2 regularization
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),  # Convolutional layer with L2 regularization
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.MaxPooling2D((2, 2)),  # Max pooling
    tf.keras.layers.Dropout(0.5),  # Dropout layer

    # Fully Connected Block
    tf.keras.layers.Flatten(),  # Flatten the 3D output to 1D tensor
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Fully connected layer with L2 regularization
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.Dropout(0.5),  # Dropout layer

    # Output Layer
    tf.keras.layers.Dense(10, activation='softmax')  # A softmax layer with 10 output units (one per class)
])

# Compile the model with Adam optimizer and categorical cross-entropy loss function.
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Train the model for 100 epochs with a batch size of 64.
# Use the test set for validation and apply the callbacks.
history = model.fit(train_images, train_labels,
                    epochs=100,
                    batch_size=64,
                    validation_data=(test_images, test_labels),
                    callbacks=[es, mc, reduce_lr])


Epoch 1/100
797/797 [==============================] - ETA: 0s - loss: 3.8970 - accuracy: 0.1439
Epoch 1: val_accuracy improved from -inf to 0.23389, saving model to best_model.h5
797/797 [==============================] - 19s 15ms/step - loss: 3.8970 - accuracy: 0.1439 - val_loss: 2.9381 - val_accuracy: 0.2339 - lr: 1.0000e-04
Epoch 2/100
  6/797 [..............................] - ETA: 9s - loss: 3.4541 - accuracy: 0.2083 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


796/797 [============================>.] - ETA: 0s - loss: 3.2648 - accuracy: 0.2227
Epoch 2: val_accuracy improved from 0.23389 to 0.30644, saving model to best_model.h5
797/797 [==============================] - 11s 14ms/step - loss: 3.2647 - accuracy: 0.2228 - val_loss: 2.6656 - val_accuracy: 0.3064 - lr: 1.0000e-04
Epoch 3/100
795/797 [============================>.] - ETA: 0s - loss: 3.0082 - accuracy: 0.2538
Epoch 3: val_accuracy improved from 0.30644 to 0.33122, saving model to best_model.h5
797/797 [==============================] - 11s 13ms/step - loss: 3.0082 - accuracy: 0.2538 - val_loss: 2.5881 - val_accuracy: 0.3312 - lr: 1.0000e-04
Epoch 4/100
794/797 [============================>.] - ETA: 0s - loss: 2.8584 - accuracy: 0.2685
Epoch 4: val_accuracy improved from 0.33122 to 0.34778, saving model to best_model.h5
797/797 [==============================] - 11s 13ms/step - loss: 2.8586 - accuracy: 0.2684 - val_loss: 2.5294 - val_accuracy: 0.3478 - lr: 1.0000e-04
Epoch 5/100
7

In [9]:
from tensorflow.keras.models import load_model

# Load the best model
saved_model = load_model('best_model.h5')

# Evaluate the model
#test_loss, test_acc = saved_model.evaluate(test_images, test_labels)
#print(f'Test accuracy: {test_acc}')


In [10]:
"""
## My model weights are saved in 'best_model.h5'. Use this weights to test.
from tensorflow.keras import models
import pickle
import tensorflow as tf

model = models.load_model("best_model.h5")
test_labels = pickle.load(open("./636_project1_test_labels", 'rb'))
test_images = pickle.load(open("./636_project1_test_images", 'rb'))


# Include your data preprocessing code if applicable
# <your data preprocessing code>
# Include your data preprocessing code if applicable


test_loss, test_acc = model.evaluate(test_images, test_labels)
your_score = round(test_acc*1000) / 10
print(f"Your Score: {your_score}")
"""

'\nfrom tensorflow.keras import models\nimport pickle\nimport tensorflow as tf\n\nmodel = models.load_model("best_model.h5")\ntest_labels = pickle.load(open("./636_project1_test_labels", \'rb\'))\ntest_images = pickle.load(open("./636_project1_test_images", \'rb\'))\n\n\n# Include your data preprocessing code if applicable\n# <your data preprocessing code>\n# Include your data preprocessing code if applicable\n\n\ntest_loss, test_acc = model.evaluate(test_images, test_labels)\nyour_score = round(test_acc*1000) / 10\nprint(f"Your Score: {your_score}")\n'